# Notes #
1. Data preparation
    + Issues
        - Number of rows less than expected -- DONE
        - Dealing with missing values -- DONE
        - Converting target into binary 
        - Should I consider doing feature selection?
    + TODO
        - Add data exploratory steps
        - Show the data distribution etc
        - Add frequency of categorical fields
        - Add class imbalance details
2. Keras model
    - **Accuracy in the first epoch itself provides optimum results.  
    Check why this could be happening**
3. Scikit model
    - Note -- Check Keras wrapper for cross validation
4. Validation


**Rough notes**
1. scikit DONE
2. read
3. cuda
4. fix errors Later, since performance is close
5. Confusion matrix, ROC and validation

---

**Data preparation**

In [23]:
# Data preparation #

# Load CSV using Pandas
import pandas as pd
import numpy as np
# filename = 'adult.data1.csv'
names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
         'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
         'hours-per-week', 'native-country', 'target']
data = pd.read_csv("../data/dataset.csv", delimiter=" ", header=None, names=names)

# Check for columns that contain missing values #
# col_names = data.columns
# print(col_names)
# num_data = data.shape[0]
# for c in col_names:
#     num_non = data[c].isin(["?"]).sum()
#     if num_non > 0:
#         print (c)
#         print (num_non)
#         print ("{0:.2f}%".format(float(num_non) / num_data * 100))
#         print ("\n")

data = data[data["workclass"] != "?"]
data = data[data["occupation"] != "?"]
data = data[data["native-country"] != "?"]

# Convert categorical fields #
categorical_col = ['workclass', 'education', 'marital-status', 'occupation',
                   'relationship', 'race', 'sex', 'native-country', 'target']
    
# for col in categorical_col:
#     categories = unique_of(data.col)
#     num_cat = count(categories)
#     for cat in categories:
#         data.col[cat] = index_of(cat in categories)

for col in categorical_col:
    b, c = np.unique(data[col], return_inverse=True)
    data[col] = c

feature_list = names[:14]
# Test train split #
X = data.loc[:, feature_list]
Y = data[['target']]

# Split the dataset into test and train datasets

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=0)

data.shape
data.head()


(45222, 15)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,25,2,226802,1,7,4,6,3,2,1,0,0,40,38,0
1,38,2,89814,11,9,2,4,0,4,1,0,0,50,38,0
2,28,1,336951,7,12,2,10,0,4,1,0,0,40,38,1
3,44,2,160323,15,10,2,6,0,2,1,7688,0,40,38,1
5,34,2,198693,0,6,4,7,1,4,1,0,0,30,38,0


---

**Data summary**

In [24]:
data.describe()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
count,45222.000000,45222.000000,4.522200e+04,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000,45222.000000
mean,38.547941,2.204502,1.897347e+05,10.313321,10.118460,2.585180,5.969572,1.412697,3.680288,0.675048,1101.430344,88.595418,40.938017,36.401022,0.247844
std,13.217870,0.958122,1.056392e+05,3.817014,2.552881,1.500458,4.026400,1.597227,0.832353,0.468362,7506.430084,404.956092,12.007508,6.079856,0.431766
min,17.000000,0.000000,1.349200e+04,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,28.000000,2.000000,1.173882e+05,9.000000,9.000000,2.000000,2.000000,0.000000,4.000000,0.000000,0.000000,0.000000,40.000000,38.000000,0.000000
50%,37.000000,2.000000,1.783160e+05,11.000000,10.000000,2.000000,6.000000,1.000000,4.000000,1.000000,0.000000,0.000000,40.000000,38.000000,0.000000
75%,47.000000,2.000000,2.379260e+05,12.000000,13.000000,4.000000,9.000000,3.000000,4.000000,1.000000,0.000000,0.000000,45.000000,38.000000,0.000000
max,90.000000,6.000000,1.490400e+06,15.000000,16.000000,6.000000,13.000000,5.000000,4.000000,1.000000,99999.000000,4356.000000,99.000000,40.000000,1.000000


In [25]:
print("Class 1 distribution -- ", data['target'].value_counts()[0] / data.shape[0] * 100)
print("Class 2 distribution -- ", data['target'].value_counts()[1] / data.shape[0] * 100)

Class 1 distribution --  75.21560302507629
Class 2 distribution --  24.78439697492371


In [26]:
# Field distribution #
category_col =['workclass', 'race', 'education','marital-status', 'occupation',
               'relationship', 'sex', 'native-country', 'target'] 
for c in category_col:
    print (c)
    print (data[c].value_counts())

workclass
2    33307
4     3796
1     3100
5     1946
3     1646
0     1406
6       21
Name: workclass, dtype: int64
race
4    38903
2     4228
1     1303
0      435
3      353
Name: race, dtype: int64
education
11    14783
15     9899
9      7570
12     2514
8      1959
1      1619
7      1507
0      1223
5       823
14      785
6       676
2       577
10      544
4       449
3       222
13       72
Name: education, dtype: int64
marital-status
2    21055
4    14598
0     6297
5     1411
6     1277
3      552
1       32
Name: marital-status, dtype: int64
occupation
2     6020
9     6008
3     5984
0     5540
11    5408
7     4808
6     2970
13    2316
5     2046
4     1480
12    1420
10     976
8      232
1       14
Name: occupation, dtype: int64
relationship
0    18666
1    11702
3     6626
4     4788
5     2091
2     1349
Name: relationship, dtype: int64
sex
1    30527
0    14695
Name: sex, dtype: int64
native-country
38    41292
25      903
29      283
10      193
32      175
1     

---

**Keras Model**

In [29]:
# Keras #

### Logistic regression using DNN ###
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils

np.random.seed(7)

# Defining model #

model = Sequential()
model.add(Dense(1,input_dim=14,activation='sigmoid'))

# Compile the model #

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model #

model.fit(x_train, y_train, epochs=10, batch_size=100)

# Evaluate the model #

scores = model.evaluate(x_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/10
18088/18088 [==============================] - 1s 36us/step - loss: 3.9609 - acc: 0.7543
Epoch 2/10
18088/18088 [==============================] - 1s 28us/step - loss: 3.9609 - acc: 0.7543
Epoch 3/10
18088/18088 [==============================] - 0s 25us/step - loss: 3.9609 - acc: 0.7543
Epoch 4/10
18088/18088 [==============================] - 0s 24us/step - loss: 3.9609 - acc: 0.7543
Epoch 5/10
18088/18088 [==============================] - 0s 23us/step - loss: 3.9609 - acc: 0.7543
Epoch 6/10
18088/18088 [==============================] - 0s 24us/step - loss: 3.9609 - acc: 0.7543
Epoch 7/10
18088/18088 [==============================] - 0s 25us/step - loss: 3.9609 - acc: 0.7543
Epoch 8/10
18088/18088 [==============================] - 0s 25us/step - loss: 3.9609 - acc: 0.7543
Epoch 9/10
18088/18088 [==============================] - 0s 26us/step - loss: 3.9609 - acc: 0.7543
Epoch 10/10
18088/18088 [==============================] - 1s 28us/step - loss: 3.9609 - acc: 0.7543

27134/27134 [==============================] - 1s 28us/step

acc: 75.08%


In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1)                 15        
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


In [36]:
config = model.get_config()
config

{'name': 'sequential_4',
 'layers': [{'class_name': 'Dense',
   'config': {'name': 'dense_4',
    'trainable': True,
    'batch_input_shape': (None, 14),
    'dtype': 'float32',
    'units': 1,
    'activation': 'sigmoid',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'scale': 1.0,
      'mode': 'fan_avg',
      'distribution': 'uniform',
      'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}}]}

In [37]:
model.get_weights()

[array([[ 2.27164924e-01],
        [ 2.38100886e-01],
        [-2.25130171e-01],
        [-4.44042087e-02],
        [ 1.81696415e-01],
        [-1.14962995e-01],
        [-6.29365087e-01],
        [-3.89337540e-04],
        [-4.67274457e-01],
        [-4.85572100e-01],
        [-1.66976452e-02],
        [-4.52749550e-01],
        [ 3.34248722e-01],
        [-4.44191515e-01]], dtype=float32), array([0.], dtype=float32)]

---

**Scikit model**

In [3]:
# Scikit learn #

# Import and create an instance of your model(Logistic regression)

from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()

# Train your model using the training dataset

logisticRegr.fit(x_train,y_train)

# Predict the output 

predictions = logisticRegr.predict(x_test)
print(predictions)

score = logisticRegr.score(x_test,y_test)
print(score)


C:\Users\shakk\Anaconda2\envs\py36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0 0 0 ... 1 0 0]
0.7865777253630132


---

**Kfold validation**

In [4]:
# Validation #
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
import numpy

seed = 7
numpy.random.seed(seed)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []

for train, test in kfold.split(X, Y):
    # create model
    model = Sequential()
    model.add(Dense(1,input_dim=14,activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(X.iloc[train], Y.iloc[train], epochs=500, batch_size=100, verbose=0)
    # evaluate the model
    scores = model.evaluate(X.iloc[test], Y.iloc[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))


acc: 75.22%
acc: 75.22%
acc: 75.22%
acc: 24.70%


KeyboardInterrupt: 

---

**ROC Curve and Confusion matrix**

In [ ]:
# Prediction #
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

y_pred = model.predict(x_test)
# y_pred = (y_pred>0.5)
fpr, tpr, threshold = metrics.roc_curve(y_test, y_pred)
roc_auc = metrics.auc(fpr, tpr)


plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


# Creating the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

---

**Classification Report**

In [ ]:
from sklearn.metrics import classification_report

target_names = ['<=50K', '>50']
print(classification_report(y_test, y_pred, target_names=target_names))

---

# Pending #
1. See if you can fine tune keras model(75% acc) to match scikit model(78% acc)
2. Exploratory analysis of the dataset
3. Latex explanations
4. **Cross check what's wrong with ROC  
and Confusion matrix**